In [1]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.3.1


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 9s 1us/step


In [3]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28)/255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

In [8]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[tf.metrics.SparseCategoricalAccuracy()])
    
    return model

In [9]:
model = create_model()

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [11]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model.fit(train_images, train_labels, epochs = 10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Epoch 1/10
15/32 [=============>................] - ETA: 0s - loss: 1.6135 - sparse_categorical_accuracy: 0.5250
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 19ms/step - loss: 1.1884 - sparse_categorical_accuracy: 0.6540 - val_loss: 0.7699 - val_sparse_categorical_accuracy: 0.7600
Epoch 2/10
18/32 [===============>..............] - ETA: 0s - loss: 0.5091 - sparse_categorical_accuracy: 0.8559
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.4408 - sparse_categorical_accuracy: 0.8770 - val_loss: 0.5341 - val_sparse_categorical_accuracy: 0.8330
Epoch 3/10
14/32 [============>.................] - ETA: 0s - loss: 0.3170 - sparse_categorical_accuracy: 0.9107
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.2950 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.4896 - val_sparse_categorical_accuracy: 0.8460
Epoch 4/10
17

In [12]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3254 - sparse_categorical_accuracy: 0.1030
Untrained model, accuracy: 10.30%


In [13]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.3959 - sparse_categorical_accuracy: 0.8720
Restored model, accuracy: 87.20%


In [14]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 5*batch_size
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
         train_labels,
         epochs=50,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [16]:
latest

'training_2\\cp-0050.ckpt'

In [19]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4821 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


In [20]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4821 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


In [21]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1022 - sparse_categorical_accuracy: 0.6860
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4259 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2996 - sparse_categorical_accuracy: 0.9160
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2168 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1522 - sparse_categorical_accuracy: 0.9670
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [23]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4280 - sparse_categorical_accuracy: 0.8620
Restored model, accuracy: 86.20%
(1000, 10)


In [25]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1211 - sparse_categorical_accuracy: 0.6830
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4190 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2920 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1943 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1583 - sparse_categorical_accuracy: 0.9650


In [26]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4377 - sparse_categorical_accuracy: 0.8590
Restored model, accuracy: 85.90%
